# 🔌 Plugins (with semantic functions)

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    // .AddOpenAIChatCompletion(
    //     modelId: Secrets.OpenAIModel,
    //     apiKey: Secrets.OpenAIApiKey)
    .Build();

### 🦜 Let's build a plugin consisting of semantic functions.

### Now the kernel contains all plugins ever added, unless we start removing them.
Entire plugins or individual functions can be removed from the kernel anytime.

### 📋 Let's prepare the execution settings and the kernel arguments.

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 2000,
     Temperature = 0.1
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas",
    ["today"] = "15th June 2024"
};

### and import the datetime semantic plugin

In [ ]:
using System.IO;

kernel.ImportPluginFromPromptDirectory(
    Path.Combine(Directory.GetCurrentDirectory(), "../../Plugins", "DateTimeSemanticPlugin"), 
    "date_semantic");

### We just imported a new plugin, let's take a look at it.

In [ ]:
#!import ../../Helpers/PrintHelper.cs

PrintHelper.PrintAllPluginFunctions(kernel);

choose DaysTo function from the semantic plugin and invoke it

In [ ]:
var daysToFunction = kernel.Plugins["date_semantic"]["DaysTo"];

var response = await kernel.InvokeAsync(daysToFunction, kernelArguments);

display(response.GetValue<string>())

### In this example, the plugin we've added `DateTimePlugin` is used in the templated prompt. The native function GetCurrentDate is invoked to enrich the prompt context with current date.

In [ ]:
using System.ComponentModel;
public class DateTimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current date")]
    public string GetCurrentDate() => DateTime.Now.ToShortDateString();
}

### Import the DateTimePlugin (consisting of native functions)

In [ ]:
kernel.ImportPluginFromType<DateTimePlugin>("date_native");

### We have importend another plugin, let's take another look at kernel plugins.

In [ ]:
PrintHelper.PrintAllPluginFunctions(kernel);

### Create a new semantic function which is calling the native function in its prompt, then invoke the semantic function

In [ ]:

// date_native.GetCurrentDate is resolved before invoking the GPT
// is crucial that GetCurrentDate function to exist in kernel already
var promptTemplate = "Considering that today date is {{date_native.GetCurrentDate}}, how many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

### 👀 Let's check the rendered prompt.

In [ ]:
#pragma warning disable SKEXP0001
display(response.RenderedPrompt);

### 🚰 For a better user experience we can may want to use with streaming 🚰.

In [ ]:
await foreach (var token in kernel.InvokePromptStreamingAsync(promptTemplate, kernelArguments))
{
    Console.Write(token);
}

### There are many ways to build plugins in SK

### You can build them and then add them to the kernel

In [ ]:
// create plugins
// kernel.CreatePluginFromFunctions()

// kernel.CreatePluginFromObject()

// kernel.CreatePluginFromPromptDirectory()

// kernel.CreatePluginFromType<>()

// add plugins to kernel
// kernel.Plugins.Add(plugin);

### Or you can add them directly (import)...

In [ ]:
// kernel.ImportPluginFromPromptDirectory();           // semantic

// kernel.ImportPluginFromFunctions()                  // native

// kernel.ImportPluginFromObject()                     // native

// kernel.ImportPluginFromType<>()                     // native or semantic from native

// kernel.ImportPluginFromOpenAIAsync()                // semantic from OpenAI plugin